In [1]:
!conda update --all --yes
!pip install --upgrade pip
!pip install tensorflow-gpu
!pip install imutils
!pip install tqdm

Solving environment: / 
  - conda-forge::libgfortran-3.0.0-1
  - defaults::libgfortran-3.0.0done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.12
  latest version: 4.7.12

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.

Requirement already up-to-date: pip in /home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages (19.2.3)


In [2]:
!mkdir data
!curl https://ecs.victoria.ac.nz/foswiki/pub/Courses/COMP309_2019T2/Assignments/Train_data_2019.zip -o data/train.zip
!unzip -o data/train.zip -d data

mkdir: cannot create directory ‘data’: File exists
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 67.5M  100 67.5M    0     0  4344k      0  0:00:15  0:00:15 --:--:-- 5626k
Archive:  data/train.zip
  inflating: data/Train_data/strawberry/strawberry_0943.jpg  
  inflating: data/Train_data/strawberry/strawberry_1485.jpg  
  inflating: data/Train_data/strawberry/strawberry_0957.jpg  
  inflating: data/Train_data/strawberry/strawberry_1491.jpg  
  inflating: data/Train_data/strawberry/strawberry_0758.jpg  
  inflating: data/Train_data/strawberry/strawberry_1320.jpg  
  inflating: data/Train_data/strawberry/strawberry_0770.jpg  
  inflating: data/Train_data/strawberry/strawberry_1687.jpg  
  inflating: data/Train_data/strawberry/strawberry_1693.jpg  
  inflating: data/Train_data/strawberry/strawberry_1877.jpg  
  inflating: data/Train_data/strawberry/strawberry_1888.jpg  
  inf

  inflating: data/Train_data/strawberry/strawberry_1044.jpg  
  inflating: data/Train_data/strawberry/strawberry_1050.jpg  
  inflating: data/Train_data/strawberry/strawberry_0428.jpg  
  inflating: data/Train_data/strawberry/strawberry_1736.jpg  
  inflating: data/Train_data/strawberry/strawberry_0831.jpg  
  inflating: data/Train_data/strawberry/strawberry_0164.jpg  
  inflating: data/Train_data/strawberry/strawberry_2015.jpg  
  inflating: data/Train_data/strawberry/strawberry_0602.jpg  
  inflating: data/Train_data/strawberry/strawberry_0616.jpg  
  inflating: data/Train_data/strawberry/strawberry_2001.jpg  
  inflating: data/Train_data/strawberry/strawberry_0158.jpg  
  inflating: data/Train_data/strawberry/strawberry_1246.jpg  
  inflating: data/Train_data/strawberry/strawberry_1520.jpg  
  inflating: data/Train_data/strawberry/strawberry_2029.jpg  
  inflating: data/Train_data/strawberry/strawberry_1252.jpg  
  inflating: data/Train_data/strawberry/strawberry_1253.jpg  
  inflat

  inflating: data/Train_data/strawberry/strawberry_0289.jpg  
  inflating: data/Train_data/strawberry/strawberry_1197.jpg  
  inflating: data/Train_data/strawberry/strawberry_0712.jpg  
  inflating: data/Train_data/strawberry/strawberry_0074.jpg  
  inflating: data/Train_data/strawberry/strawberry_0060.jpg  
  inflating: data/Train_data/strawberry/strawberry_1418.jpg  
  inflating: data/Train_data/strawberry/strawberry_0706.jpg  
  inflating: data/Train_data/strawberry/strawberry_1430.jpg  
  inflating: data/Train_data/strawberry/strawberry_2139.jpg  
  inflating: data/Train_data/strawberry/strawberry_1356.jpg  
  inflating: data/Train_data/strawberry/strawberry_1342.jpg  
  inflating: data/Train_data/strawberry/strawberry_0935.jpg  
  inflating: data/Train_data/strawberry/strawberry_1381.jpg  
  inflating: data/Train_data/strawberry/strawberry_0089.jpg  
  inflating: data/Train_data/strawberry/strawberry_0923.jpg  
  inflating: data/Train_data/strawberry/strawberry_1383.jpg  
  inflat

  inflating: data/Train_data/strawberry/strawberry_2061.jpg  
  inflating: data/Train_data/strawberry/strawberry_1568.jpg  
  inflating: data/Train_data/strawberry/strawberry_0676.jpg  
  inflating: data/Train_data/strawberry/strawberry_0110.jpg  
  inflating: data/Train_data/strawberry/strawberry_0104.jpg  
  inflating: data/Train_data/strawberry/strawberry_0662.jpg  
  inflating: data/__MACOSX/Train_data/strawberry/._strawberry_0662.jpg  
  inflating: data/Train_data/strawberry/strawberry_2075.jpg  
  inflating: data/Train_data/strawberry/strawberry_0892.jpg  
  inflating: data/Train_data/strawberry/strawberry_1554.jpg  
  inflating: data/Train_data/strawberry/strawberry_1232.jpg  
  inflating: data/Train_data/strawberry/strawberry_0138.jpg  
  inflating: data/Train_data/strawberry/strawberry_1226.jpg  
  inflating: data/Train_data/strawberry/strawberry_2049.jpg  
  inflating: data/Train_data/strawberry/strawberry_0886.jpg  
  inflating: data/Train_data/strawberry/strawberry_1540.jpg

  inflating: data/Train_data/strawberry/strawberry_0651.jpg  
  inflating: data/Train_data/strawberry/strawberry_0137.jpg  
  inflating: data/Train_data/strawberry/strawberry_1229.jpg  
  inflating: data/Train_data/strawberry/strawberry_0650.jpg  
  inflating: data/Train_data/strawberry/strawberry_0888.jpg  
  inflating: data/Train_data/strawberry/strawberry_1228.jpg  
  inflating: data/Train_data/strawberry/strawberry_0122.jpg  
  inflating: data/Train_data/strawberry/strawberry_2053.jpg  
  inflating: data/Train_data/strawberry/strawberry_1572.jpg  
  inflating: data/Train_data/strawberry/strawberry_1214.jpg  
  inflating: data/Train_data/strawberry/strawberry_0678.jpg  
  inflating: data/Train_data/strawberry/strawberry_1566.jpg  
  inflating: data/Train_data/strawberry/strawberry_0693.jpg  
  inflating: data/Train_data/strawberry/strawberry_2084.jpg  
  inflating: data/Train_data/strawberry/strawberry_2090.jpg  
  inflating: data/Train_data/strawberry/strawberry_1599.jpg  
  inflat

...

  inflating: data/Train_data/tomato/tomato_0034.jpg  
  inflating: data/Train_data/tomato/tomato_0752.jpg  
  inflating: data/Train_data/tomato/tomato_1464.jpg  
  inflating: data/Train_data/tomato/tomato_1302.jpg  
  inflating: data/Train_data/tomato/tomato_1316.jpg  
  inflating: data/Train_data/tomato/tomato_1470.jpg  
  inflating: data/Train_data/tomato/tomato_1699.jpg  
  inflating: data/Train_data/tomato/tomato_0593.jpg  
  inflating: data/Train_data/tomato/tomato_1855.jpg  
  inflating: data/Train_data/tomato/tomato_1869.jpg  
  inflating: data/Train_data/tomato/tomato_1882.jpg  
  inflating: data/Train_data/tomato/tomato_0550.jpg  
  inflating: data/Train_data/tomato/tomato_1896.jpg  
  inflating: data/Train_data/tomato/tomato_1128.jpg  
  inflating: data/Train_data/tomato/tomato_0578.jpg  
  inflating: data/Train_data/tomato/tomato_1672.jpg  
  inflating: data/Train_data/tomato/tomato_1114.jpg  
  inflating: data/Train_data/tomato/tomato_1673.jpg  
  inflating: data/Train_data

  inflating: data/Train_data/tomato/tomato_0153.jpg  
  inflating: data/__MACOSX/Train_data/tomato/._tomato_0153.jpg  
  inflating: data/Train_data/tomato/tomato_0147.jpg  
  inflating: data/__MACOSX/Train_data/tomato/._tomato_0147.jpg  
  inflating: data/Train_data/tomato/tomato_1259.jpg  
  inflating: data/Train_data/tomato/tomato_0621.jpg  
  inflating: data/Train_data/tomato/tomato_1517.jpg  
  inflating: data/Train_data/tomato/tomato_0609.jpg  
  inflating: data/Train_data/tomato/tomato_1271.jpg  
  inflating: data/Train_data/tomato/tomato_1265.jpg  
  inflating: data/Train_data/tomato/tomato_1503.jpg  
  inflating: data/Train_data/tomato/tomato_0190.jpg  
  inflating: data/Train_data/tomato/tomato_0184.jpg  
  inflating: data/Train_data/tomato/tomato_0812.jpg  
  inflating: data/Train_data/tomato/tomato_0806.jpg  
  inflating: data/Train_data/tomato/tomato_1729.jpg  
  inflating: data/Train_data/tomato/tomato_0345.jpg  
  inflating: data/Train_data/tomato/tomato_1073.jpg  
  infl

  inflating: data/Train_data/tomato/tomato_1097.jpg  
  inflating: data/Train_data/tomato/tomato_1929.jpg  
  inflating: data/Train_data/tomato/tomato_1903.jpg  
  inflating: data/Train_data/tomato/tomato_1917.jpg  
  inflating: data/Train_data/tomato/tomato_1081.jpg  
  inflating: data/Train_data/tomato/tomato_1095.jpg  
  inflating: data/Train_data/tomato/tomato_0360.jpg  
  inflating: data/Train_data/tomato/tomato_1718.jpg  
  inflating: data/Train_data/tomato/tomato_0412.jpg  
  inflating: data/Train_data/tomato/tomato_1042.jpg  
  inflating: data/Train_data/tomato/tomato_1730.jpg  
  inflating: data/Train_data/tomato/tomato_0348.jpg  
  inflating: data/Train_data/tomato/tomato_1056.jpg  
  inflating: data/Train_data/tomato/tomato_0823.jpg  
  inflating: data/Train_data/tomato/tomato_0189.jpg  
  inflating: data/Train_data/tomato/tomato_0837.jpg  
  inflating: data/Train_data/tomato/tomato_0604.jpg  
  inflating: data/Train_data/tomato/tomato_0162.jpg  
  inflating: data/Train_data

  inflating: data/Train_data/tomato/tomato_1858.jpg  
  inflating: data/Train_data/tomato/tomato_0777.jpg  
  inflating: data/Train_data/tomato/tomato_1469.jpg  
  inflating: data/Train_data/tomato/tomato_0005.jpg  
  inflating: data/Train_data/tomato/tomato_0763.jpg  
  inflating: data/Train_data/tomato/tomato_1455.jpg  
  inflating: data/Train_data/tomato/tomato_0993.jpg  
  inflating: data/Train_data/tomato/tomato_1333.jpg  
  inflating: data/Train_data/tomato/tomato_1327.jpg  
  inflating: data/Train_data/tomato/tomato_0039.jpg  
  inflating: data/Train_data/tomato/tomato_0987.jpg  
  inflating: data/Train_data/tomato/tomato_0978.jpg  
  inflating: data/Train_data/tomato/tomato_1496.jpg  
  inflating: data/Train_data/tomato/tomato_0950.jpg  
  inflating: data/Train_data/tomato/tomato_0788.jpg  
  inflating: data/Train_data/tomato/tomato_1482.jpg  
  inflating: data/Train_data/tomato/tomato_0944.jpg  
  inflating: data/Train_data/tomato/tomato_0603.jpg  
  inflating: data/Train_data

  inflating: data/Train_data/tomato/tomato_1975.jpg  
  inflating: data/Train_data/tomato/tomato_1959.jpg  
  inflating: data/__MACOSX/Train_data/tomato/._tomato_1959.jpg  
  inflating: data/Train_data/tomato/tomato_1781.jpg  
  inflating: data/Train_data/tomato/tomato_1795.jpg  
  inflating: data/Train_data/tomato/tomato_1965.jpg  
  inflating: data/Train_data/tomato/tomato_1971.jpg  
  inflating: data/Train_data/tomato/tomato_1742.jpg  
  inflating: data/Train_data/tomato/tomato_1756.jpg  
  inflating: data/Train_data/tomato/tomato_0448.jpg  
  inflating: data/Train_data/tomato/tomato_1030.jpg  
  inflating: data/Train_data/tomato/tomato_1018.jpg  
  inflating: data/Train_data/tomato/tomato_0460.jpg  
  inflating: data/Train_data/tomato/tomato_0474.jpg  
  inflating: data/Train_data/tomato/tomato_0845.jpg  
  inflating: data/Train_data/tomato/tomato_1597.jpg  
  inflating: data/Train_data/tomato/tomato_0851.jpg  
  inflating: data/Train_data/tomato/tomato_0689.jpg  
  inflating: data

In [1]:
%matplotlib inline

import os
os.environ["CUDA_VISIBLE_DEVICES"]="-1"  

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import os
import numpy as np
import matplotlib.pyplot as plt
import glob
import imageio
from tqdm import tqdm_notebook as tqdm

In [2]:
train_dir = 'data/Train_data'

epochs = 15
IMG_HEIGHT = 300
IMG_WIDTH = 300
train_pct = 0.8

In [3]:
labels = []
images = []

for label_dir in glob.glob(f'{train_dir}/*'):
    label = label_dir.split('/')[-1]
    for image_path in tqdm(glob.glob(f'{label_dir}/*'), label):
        labels += [label]
        images += [imageio.imread(image_path)]

In [4]:
y = np.zeros((len(labels)))
x = np.zeros((len(images), IMG_HEIGHT, IMG_WIDTH, 3))

for i, (label, image) in tqdm(list(enumerate(zip(labels, images)))):
    if image.shape != (300, 300, 3): continue
    y[i] = ['cherry', 'strawberry', 'tomato'].index(label)
    x[i, :, :] = image

In [5]:
np.random.seed(309)
is_train = (np.random.random((len(labels))) <= train_pct)

y_train = y[is_train]
y_test = y[~is_train]

x_train = x[is_train, :, :, :]
x_test = x[~is_train, :, :, :]

In [6]:
model = Sequential([
    Conv2D(16, 3, padding='same', activation='relu', input_shape=(IMG_HEIGHT, IMG_WIDTH ,3)),
    MaxPooling2D(),
    Conv2D(32, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Conv2D(64, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Flatten(),
    Dense(512, activation='relu'),
    Dense(1, activation='sigmoid')
])

model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

In [7]:
r = model.fit(x_train, y_train, epochs=25, validation_data=(x_test, y_test))

Train on 3559 samples, validate on 941 samples
Epoch 1/25
 512/3559 [===>..........................] - ETA: 3:02 - loss: 310.4081 - accuracy: 0.3521

KeyboardInterrupt: 